# Калибровка

In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
import json
import random
from tqdm.notebook import trange

## Константы

_cam1_ - левая камера  
_cam2_ - правая камера

In [ ]:
SQUARE_SIZE = (0.01, 0.01)  # size of chess square (in m)
CHESSBOARD_SIZE = (7, 7)
camera_params = {
    'focal_length_m': 80e-3,
    'img_resolution': (5120, 4096),
    'pixel_sz_x': 4.5e-6,  # in m
    'pixel_sz_y': 4.5e-6  # in m
}
stereo_params = {
    'baseline': 412e-3,  # in m
    'center_dist_to_obj': 639e-3,  # in m
}

## Данные

Данные сгенерированы на основе скрипта `render_chessboard.py`.  
Парные изображения расположены в соответствующей папке `chess_dir/`, каждая пара в папке `pair_*/`. 

In [ ]:
imgs_folder = '/home/deniskirbaba/AI Talent Hub/Product Hack - fall 2024/chess_dir/'

pair_dirs = glob.glob(f"{imgs_folder}/pair_*/")

cam1_imgs = {}
cam2_imgs = {}

for i in trange(20, desc="Loading Image Pairs"):
    pair_dir = f"{imgs_folder}/pair_{i}/"
    
    left_img_path = os.path.join(pair_dir, 'image_left.png')
    right_img_path = os.path.join(pair_dir, 'image_right.png')
    
    if os.path.exists(left_img_path):
        img_name = f"pair_{i}_left"
        img = cv.imread(left_img_path, cv.IMREAD_GRAYSCALE)
        cam1_imgs[img_name] = img
    
    if os.path.exists(right_img_path):
        img_name = f"pair_{i}_right"
        img = cv.imread(right_img_path, cv.IMREAD_GRAYSCALE)
        cam2_imgs[img_name] = img

In [ ]:
# Remove bad calibration imgs
del cam1_imgs['pair_6_left']
del cam1_imgs['pair_7_left']
del cam2_imgs['pair_6_right']
del cam2_imgs['pair_7_right']

In [ ]:
random_keys = random.sample(list(cam1_imgs.keys()), 3)

fig, axes = plt.subplots(3, 2, figsize=(10, 10))

for i, key in enumerate(random_keys):
    left_img = cam1_imgs[key]
    
    right_img_key = key.replace('left', 'right')
    right_img = cam2_imgs.get(right_img_key, None)
    
    axes[i, 0].imshow(left_img, cmap='gray')
    axes[i, 0].set_title(f"Left: {key}")
    axes[i, 0].axis('off')
    
    if right_img is not None:
        axes[i, 1].imshow(right_img, cmap='gray')
        axes[i, 1].set_title(f"Right: {right_img_key}")
        axes[i, 1].axis('off')
    else:
        axes[i, 1].axis('off')

plt.tight_layout()
plt.show()

## Поиск внутренних параметров камер

### Вычисление camera matrix

Camera matrix можно вычислить исходя из следующих параметров:
1. фокусное расстояние: 80mm
2. размер пикселя: 4.5 µm × 4.5 µm
3. разрешение изображения: 5120 × 4096 

In [ ]:
def calculate_CM(focal_length_m, img_resolution, pixel_sz_x, pixel_sz_y):
    width, height = img_resolution

    f_x = focal_length_m / pixel_sz_x
    f_y = focal_length_m / pixel_sz_y

    c_x = width / 2.0
    c_y = height / 2.0

    CM = np.array([
        [f_x, 0, c_x],
        [0, f_y, c_y],
        [0, 0, 1]
    ])

    return CM

In [ ]:
CM = calculate_CM(**camera_params)
CM

In [ ]:
dist = np.zeros((5,), dtype=np.float32)

### Поиск всех внутренних параметров с помощью калибровки шахматной доской

In [ ]:
def calibrate_camera(imgs, img_resolution, 
                     square_size, chessboard_size, 
                     CM_guess=None, dist_guess=None):
    criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
    
    objp = np.zeros((chessboard_size[0] * chessboard_size[1], 3), np.float32)
    objp[:, :2] = np.mgrid[0:chessboard_size[0], 0:chessboard_size[1]].T.reshape(-1, 2)
    objp *= square_size[0] 

    objpoints = []  # 3D points in real-world space
    imgpoints = []  # 2D points in image plane

    for i in trange(len(imgs), desc='Processing Images'):
        img_key = list(imgs.keys())[i] 
        img = imgs[img_key].copy()
        
        ret, corners = cv.findChessboardCorners(img, chessboard_size, None)
        
        if ret:
            objpoints.append(objp) 
            corners = cv.cornerSubPix(img, corners, (11, 11), (-1, -1), criteria)
            imgpoints.append(corners)
            # cv.drawChessboardCorners(img, chessboard_size, corners, ret)
            # cv.imwrite(img_key+'.jpg', img)

    flags = cv.CALIB_ZERO_TANGENT_DIST + cv.CALIB_FIX_K1 + cv.CALIB_FIX_K2 + cv.CALIB_FIX_K3
    ret, CM, dist, _, _ = cv.calibrateCamera(
        objpoints, imgpoints, img_resolution, CM_guess, dist_guess, flags=flags
    )
    print(f'RMSE: {ret}')
    return CM, dist

In [ ]:
CM1, dist1 = calibrate_camera(cam1_imgs, camera_params['img_resolution'], 
                              SQUARE_SIZE, CHESSBOARD_SIZE, 
                              CM, np.zeros((5,), dtype=np.float32))
CM2, dist2 = calibrate_camera(cam2_imgs, camera_params['img_resolution'], 
                              SQUARE_SIZE, CHESSBOARD_SIZE, 
                              CM, np.zeros((5,), dtype=np.float32))

In [ ]:
CM1, dist1, CM2, dist2

## Стерео-калибровка

### Manual estimation of R, T matrices given the organizator info

In [ ]:
def create_rotation_matrix(rx_deg, ry_deg, rz_deg):
    rx_rad = np.deg2rad(rx_deg)
    ry_rad = np.deg2rad(ry_deg)
    rz_rad = np.deg2rad(rz_deg)
    
    R_x = np.array([
        [1, 0, 0],
        [0, np.cos(rx_rad), -np.sin(rx_rad)],
        [0, np.sin(rx_rad), np.cos(rx_rad)]
    ])
    R_y = np.array([
        [np.cos(ry_rad), 0, np.sin(ry_rad)],
        [0, 1, 0],
        [-np.sin(ry_rad), 0, np.cos(ry_rad)]
    ])
    R_z = np.array([
        [np.cos(rz_rad), -np.sin(rz_rad), 0],
        [np.sin(rz_rad), np.cos(rz_rad), 0],
        [0, 0, 1]
    ])
    
    R = np.matmul(np.matmul(R_z, R_y), R_x)
    
    return R

In [ ]:
# Calculating R
# Set global coordinate system in between cameras
R1 = create_rotation_matrix(0, 18.8, 0)
R2 = create_rotation_matrix(0, -18.8, 0)

R = np.dot(R2, R1.T)

# Calculating T
C1 = np.array([-0.206, 0.0, 0.0]) # Center of cam1 coord system
C2 = np.array([0.206, 0.0, 0.0])  # Center of cam2 coord system

# Translation vector
t12_world = C2 - C1

T = np.dot(R1.T, t12_world)

R, T

### Calibration using OpenCV

In [ ]:
def stereo_calibrate(cam1_imgs, cam2_imgs, img_resolution, 
                             square_size, chessboard_size, 
                             CM1, dist1, CM2, dist2, 
                             R_guess=None, T_guess=None):
    criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 100, 0.0001)
    
    objp = np.zeros((chessboard_size[0] * chessboard_size[1], 3), np.float32)
    objp[:, :2] = np.mgrid[0:chessboard_size[0], 0:chessboard_size[1]].T.reshape(-1, 2)
    objp *= square_size[0]
    
    objpoints = []  # 3D points in real-world space
    imgpoints1 = []  # 2D points in image plane of cam1 (left)
    imgpoints2 = []  # 2D points in image plane of cam2 (right)
    
    for i in trange(len(cam1_imgs), desc='Processing Stereo Image Pairs'):
        img_key = list(cam1_imgs.keys())[i] 
        img1 = cam1_imgs[img_key] 
        img2 = cam2_imgs[img_key.replace("_left", "_right")] 

        ret1, corners1 = cv.findChessboardCorners(img1, chessboard_size, None)
        ret2, corners2 = cv.findChessboardCorners(img2, chessboard_size, None)

        if ret1 and ret2:
            objpoints.append(objp) 
            
            corners1 = cv.cornerSubPix(img1, corners1, (11, 11), (-1, -1), criteria)
            corners2 = cv.cornerSubPix(img2, corners2, (11, 11), (-1, -1), criteria)
            
            imgpoints1.append(corners1) 
            imgpoints2.append(corners2)

    stereocalibration_flags = cv.CALIB_FIX_INTRINSIC# + cv.CALIB_USE_EXTRINSIC_GUESS

    RMSE, CM1, dist1, CM2, dist2, R, T, E, F, _, _, _ = cv.stereoCalibrateExtended(
        objpoints, imgpoints1, imgpoints2, 
        CM1, dist1, CM2, dist2, img_resolution, 
        R_guess, T_guess, 
        flags=stereocalibration_flags,
        criteria=criteria
    )

    print(f'Stereo Calibration RMSE: {RMSE}')
    return R, T, E, F

R: The relative rotation matrix between the first and second cameras. This matrix describes how the second camera is rotated relative to the first camera.

T: The translation vector between the first and second cameras. This vector describes how far and in which direction the second camera is from the first camera.

E: The essential matrix. This matrix encodes the rotation and translation between the two cameras, and it can be used to compute the epipolar geometry (constraints for stereo matching).

F: The fundamental matrix. This is another way of representing the geometric relationship between the two cameras. It defines the epipolar lines in stereo matching.

In [ ]:
R, T, E, F = stereo_calibrate(cam1_imgs, cam2_imgs, camera_params['img_resolution'], 
                                            SQUARE_SIZE, CHESSBOARD_SIZE, 
                                            CM1, dist1, CM2, dist2, 
                                            R, T)

print("Rotation Matrix:\n", R)
print("Translation Vector:\n", T)

In [ ]:
R, T

Проверим baseline и углы поворота для полученных матриц

In [ ]:
def rotation_matrix_to_euler_angles(R):
    sy = np.sqrt(R[0, 0] ** 2 + R[1, 0] ** 2)
    
    singular = sy < 1e-6
    
    if not singular:
        yaw = np.arctan2(R[2, 1], R[2, 2])
        pitch = np.arctan2(-R[2, 0], sy)
        roll = np.arctan2(R[1, 0], R[0, 0])
    else:
        yaw = np.arctan2(-R[1, 2], R[1, 1])
        pitch = np.arctan2(-R[2, 0], sy)
        roll = 0

    return np.degrees(np.array([yaw, pitch, roll]))

def show_baseline_and_rotation(T_calculated, R_calculated, real_baseline, R_real=None):
    calculated_baseline = np.linalg.norm(T)
    
    print(f"Calculated baseline: {calculated_baseline:.2f} m")
    print(f"Real baseline: {real_baseline:.2f} m")
    
    baseline_difference = np.abs(calculated_baseline - real_baseline)
    print(f"Baseline difference: {baseline_difference:.2f} m")
    
    euler_angles_calculated = rotation_matrix_to_euler_angles(R_calculated)
    print("\nCalculated Euler angles (Yaw, Pitch, Roll):")
    print(f"Yaw: {euler_angles_calculated[0]:.2f} degrees")
    print(f"Pitch: {euler_angles_calculated[1]:.2f} degrees")
    print(f"Roll: {euler_angles_calculated[2]:.2f} degrees")

In [ ]:
show_baseline_and_rotation(T, R, stereo_params['baseline'])

## Export the camera's intrinsic and extrinsic parameters

In [ ]:
calibration_data_json = {
    'CM': CM.tolist(),
    'dist': dist.tolist(),
    'R': R.tolist(),
    'T': T.tolist(),
    'E': E.tolist(),
    'F': F.tolist()
}

with open('data/calibration_data_synthetic.json', 'w') as f:
    json.dump(calibration_data_json, f, indent=4)